In [1]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [2]:
train_data = torch.load('/project/annotation/train_v2.pt')
test_data = torch.load('/project/annotation/test_v2.pt')
valid_data = torch.load('/project/annotation/valid_v2.pt')

# dataloaders
batch_size = 32
device='cuda'

# make sure to SHUFFLE for your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [3]:
import torch.nn as nn

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [10]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)
        

In [11]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [12]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [13]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
        
    print(np.mean(acc_list))

In [14]:
predict(model, test_loader)

0.9993296731358529
